In [ ]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

# Connecting to Cassandra
cloud_config = {
    'secure_connect_bundle': 'secure-connect-mydatabase.zip'
}
auth_provider = PlainTextAuthProvider('CLIENT_ID', 'CLIENT_SECRET')
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

# Creating a Keyspace
session.execute("CREATE KEYSPACE IF NOT EXISTS mykeyspace WITH replication = {'class': 'SimpleStrategy', 'replication_factor': '1'};")

# Using the Keyspace
session.set_keyspace('mykeyspace')

# Creating a Table
session.execute("""
    CREATE TABLE IF NOT EXISTS people (
        id UUID PRIMARY KEY,
        first_name TEXT,
        last_name TEXT,
        age INT,
        address TEXT
    );
""")

# Inserting Data
session.execute("""
    INSERT INTO people (id, first_name, last_name, age, address) VALUES
        (uuid(), 'John', 'Doe', 30, '123 Main St'),
        (uuid(), 'Jane', 'Smith', 25, '456 Elm St'),
        (uuid(), 'Alice', 'Johnson', 35, '789 Oak St');
""")

# Selecting Data
rows = session.execute("SELECT * FROM people;")
for row in rows:
    print(row.id, row.first_name, row.last_name, row.age, row.address)

# Updating Data
session.execute("UPDATE people SET age = 31 WHERE first_name = 'John' AND last_name = 'Doe';")

# Deleting Data
session.execute("DELETE FROM people WHERE first_name = 'Alice';")

# Using Indexes
session.execute("CREATE INDEX IF NOT EXISTS idx_age ON people(age);")
rows = session.execute("SELECT * FROM people WHERE age > 30;")
for row in rows:
    print(row.id, row.first_name, row.last_name, row.age, row.address)

# Using Batch Operations
batch = """
    BEGIN BATCH
    INSERT INTO people (id, first_name, last_name, age, address) VALUES (uuid(), 'Bob', 'Brown', 40, '321 Pine St');
    DELETE FROM people WHERE first_name = 'Jane';
    APPLY BATCH;
"""
session.execute(batch)

# Using Materialized Views
session.execute("""
    CREATE MATERIALIZED VIEW IF NOT EXISTS people_by_age AS
        SELECT id, first_name, last_name, age, address
        FROM people
        WHERE age IS NOT NULL
        PRIMARY KEY (age, id);
""")
rows = session.execute("SELECT * FROM people_by_age WHERE age = 40;")
for row in rows:
    print(row.id, row.first_name, row.last_name, row.age, row.address)

# Using User-Defined Types (UDTs)
session.execute("""
    CREATE TYPE IF NOT EXISTS address (
        street TEXT,
        city TEXT,
        state TEXT,
        zip_code TEXT
    );
""")
session.execute("""
    CREATE TABLE IF NOT EXISTS people (
        id UUID PRIMARY KEY,
        first_name TEXT,
        last_name TEXT,
        age INT,
        address FROZEN<address>
    );
""")
